Author: Daniel Low

Based on tutorials:
- https://github.com/huggingface/notebooks/blob/e1983033bf88432e1e371996e1deec2f6ef1c52a/examples/text_classification.ipynb
- https://huggingface.co/blog/ray-tune
- https://huggingface.co/docs/transformers/training

In [ ]:
# python and package Versions used

!python --version # I ran on python==3.10.12

Python 3.10.12


In [ ]:
# !pip install -q torch==2.0.1 datasets==2.14.3 transformers==4.28.1 accelerate==0.15.0 optuna==3.2.0 evaluate
#imbalanced-learn==0.11.0



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Print number of GPUs available
    print("Number of GPUs available:", torch.cuda.device_count())

    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}:")
        print(f"\tName: {torch.cuda.get_device_name(i)}")
        print(f"\tCompute Capability: {torch.cuda.get_device_capability(i)}")
        print(f"\tTotal Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9} GB")
        print(torch.cuda.get_device_properties(i))
        # Additional details can be accessed via `torch.cuda.get_device_properties(i)`
else:
    print("CUDA is not available. Please check your installation and if your hardware supports CUDA.")


Number of GPUs available: 1
GPU 0:
	Name: Tesla T4
	Compute Capability: (7, 5)
	Total Memory: 15.835660288 GB
_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


In [ ]:
'''
Authors: Daniel M. Low
License: See license in github repository
'''

import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

pd.set_option("display.max_columns", None)
# pd.options.display.width = 0


location = 'openmind' 



if location == 'colab':
  from google.colab import drive
  project_name = 'concept_tracker'
  drive.mount('/content/drive')
  input_dir = f'/content/drive/MyDrive/datum/{project_name}/data/ctl/'
  output_dir = f'/content/drive/MyDrive/datum/{project_name}/data/output/lexicon_paper/'
elif location == 'openmind':
  input_dir = '/nese/mit/group/sig/projects/dlow/ctl/'
  output_dir = 'home/dlow/'
elif location =='local':
  input_dir = './data/ctl/'
  output_dir = '/home/dlow/datum/lexicon/data/output/'

os.makedirs(output_dir, exist_ok=True)


Mounted at /content/drive


In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
from datasets import load_dataset, load_metric
from datasets import list_metrics
from imblearn.over_sampling import RandomOverSampler


# Fine-tuning a model on a text classification task

In [ ]:
toy = False
regression = True # False for classification
task = "suicide_risk"
text_col = 'text'
y_col = 'y'
model_checkpoint = "roberta-base" # "distilbert-base-uncased"
num_labels = 1 # Use 1 for regression
metrics_to_report = ['Model',  'RMSE','RMSE per value','MAE','MAE per value', 'rho', 'gridsearch', 'Best parameters']
metric_name = "rmse"
training_set_n = 'all' #{'all', 150, 300}
n = training_set_n
validation_key = 'validation'
imbalanced = False # if True, will use custom function. False = leave imabalance or balance through oversampling/undersampling/etc

### training or hyperparameter search arguments

In [ ]:

# if finetuning without hypeparameter search
batch_size = 16 #not higher due to GPU memory usage limit issues
if toy:
    epochs = 1
else:
    epochs = 3 #  used only if not hyperparameter search


# hyperparameter search args
do_hyperparameter_search = True
hyperparameter_shards = False #10 # 10 means you use 10th of the data from hyperparameter search
hyperparameter_search_n_trials = 10
hyperparameter_search_direction = 'minimize' # maximize for classification metric, minimize for loss

In [ ]:
# if searching with hyperparameter search, options are default of optuna or specified here:


# default optuna decided by transformers: https://github.com/huggingface/transformers/blob/dcbfd93d7aeb14f8ff08a48866d2a68950d4c69a/src/transformers/trainer_utils.py#L248

# https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/002_configurations.html#sphx-glr-tutorial-10-key-features-002-configurations-py

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1,5, step=1),
        "weight_decay": trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)
    }


## Loading the dataset

In [ ]:
balanced_validation_set = False #False: setting it to True had very bad performance). It uses 20% the training set as balanced validation, similar to other models that do 5-fold CV


train = pd.read_csv(input_dir+'train10_train_30perc_text_y_balanced_regression.csv', index_col=0)
val = pd.read_csv(input_dir+'train10_val_15perc_text_y_regression.csv', index_col=0)
test = pd.read_csv(input_dir+'train10_test_15perc_text_y_regression.csv', index_col=0)



print(train.shape, val.shape, test.shape)

if balanced_validation_set:
  from sklearn.model_selection import train_test_split
  train, val = train_test_split(train, test_size=0.2, random_state=42)
  print(train.shape, val.shape, test.shape)

(5654, 3) (5353, 3) (5353, 3)


In [ ]:


train = train.dropna()
val = val.dropna()
test = test.dropna()

if training_set_n != 'all':
  train = train.sample(training_set_n, random_state = 42)




from collections import Counter

print(train.shape, test.shape)

if toy:
  train = train.sample(frac = 0.1)
  val = val.sample(frac = 0.1)
  test = test.sample(frac = 0.1)


X_train = train[text_col].values
y_train = train[y_col].values

print('train labels:', Counter(y_train))

X_val = val[text_col].values
y_val = val[y_col].values

X_test = test[text_col].values
y_test = test[y_col].values

# Oversample to match method of model with metadata approach
# ros = RandomOverSampler(random_state=0)
# X_train, y_train = ros.fit_resample(X_train.reshape(-1, 1), y_train)
# X_train = X_train.flatten()

print(X_train.shape, y_train.shape)

d = {'train':Dataset.from_dict({'label':y_train,'text':X_train}),
     'validation':Dataset.from_dict({'label':y_val,'text':X_val}),
     'test':Dataset.from_dict({'label':y_test,'text':X_test})
     }




dataset = DatasetDict(d)


(5654, 3) (5353, 3)
train labels: Counter({1.0: 1887, 2.0: 1884, 3.0: 1883})
(5654,) (5654,)


In [ ]:
dataset['train']['label'][:10]

[2.0, 2.0, 2.0, 1.0, 3.0, 3.0, 1.0, 1.0, 1.0, 1.0]

In [ ]:
from evaluate import load


metrics_list = list_metrics()
print(len(metrics_list))
print(metrics_list)
if regression:
  metric = load('mse')
else:
  metric = load_metric("f1")



<ipython-input-12-f75b8a59469e>:4: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


221
['accuracy', 'bertscore', 'bleu', 'bleurt', 'brier_score', 'cer', 'character', 'charcut_mt', 'chrf', 'code_eval', 'comet', 'competition_math', 'confusion_matrix', 'coval', 'cuad', 'exact_match', 'f1', 'frugalscore', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis', 'mape', 'mase', 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'nist_mt', 'pearsonr', 'perplexity', 'poseval', 'precision', 'r_squared', 'recall', 'rl_reliability', 'roc_auc', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'smape', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'ter', 'trec_eval', 'wer', 'wiki_split', 'xnli', 'xtreme_s', 'AlhitawiMohammed22/CER_Hu-Evaluation-Metrics', 'BucketHeadP65/confusion_matrix', 'BucketHeadP65/roc_curve', 'CZLC/rouge_raw', 'DaliaCaRo/accents_unplugged_eval', 'DarrenChensformer/eval_keyphrase', 'DarrenChensformer/relation_extraction', 'DoctorSlimm/bangalore_score', 'DoctorSlimm/kaushiks_criteria', 'Drunper/metrica_tesi', 'Felipehonorato/eer', 'Fritz02/execution_ac

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [ ]:


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
# show_random_elements(dataset["train"], num_examples=3)

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [ ]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'mse': 0.4375}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2, 2, 2409, 42, 3645, 1411, 19, 24, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [ ]:

task_to_keys = {
    'stb': ("text", None),
    'suicide_risk': ("text", None),
}

We can double check it does work on our current dataset:

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]



if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Hello. I feel so worthless. I want all this pain to be over. I'm so tired of trying. I don't see a reason to keep going. Kinda. I want to die, I really see no reason to keep going. I know people would be sad if I died though. I don't know why they would be sad. Yeah. I want to take pills but my mom hid all of them. I've been hospitalized 2 times for suicide attempts already. Thank you. I feel so depressed. I don't think anything will get better anymore. I don't feel brave. Thank you very much. I don't really know :/. Hopelessness. I just feel like there's no reason to be happy anymore and I should just let myself fall. Also I have plans for 2017 and my mom always says I don't deserve any of it. She always says I'm faking my depression too. She tries but no a lot and thank you. No one anymore really. Thank you so so much. I'm feeling better, thank you so much. Remember that you are also loved and people care. Have an amazing night! Y.


In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]

preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 31414, 4, 38, 619, 98, 34858, 4, 38, 236, 70, 42, 2400, 7, 28, 81, 4, 38, 437, 98, 7428, 9, 667, 4, 38, 218, 75, 192, 10, 1219, 7, 489, 164, 4, 17161, 102, 4, 38, 236, 7, 1597, 6, 38, 269, 192, 117, 1219, 7, 489, 164, 4, 38, 216, 82, 74, 28, 5074, 114, 38, 962, 600, 4, 38, 218, 75, 216, 596, 51, 74, 28, 5074, 4, 8976, 4, 38, 236, 7, 185, 13866, 53, 127, 3795, 20119, 70, 9, 106, 4, 38, 348, 57, 12399, 132, 498, 13, 4260, 3881, 416, 4, 3837, 47, 4, 38, 619, 98, 16658, 4, 38, 218, 75, 206, 932, 40, 120, 357, 5988, 4, 38, 218, 75, 619, 10025, 4, 3837, 47, 182, 203, 4, 38, 218, 75, 269, 216, 4832, 11665, 13583, 13802, 1825, 4, 38, 95, 619, 101, 89, 18, 117, 1219, 7, 28, 1372, 5988, 8, 38, 197, 95, 905, 2185, 1136, 4, 1578, 38, 33, 708, 13, 193, 8, 127, 3795, 460, 161, 38, 218, 75, 6565, 143, 9, 24, 4, 264, 460, 161, 38, 437, 856, 7520, 127, 6943, 350, 4, 264, 5741, 53, 117, 10, 319, 8, 3392, 47, 4, 440, 65, 5988, 269, 4, 3837, 47, 98, 98, 203, 4, 38, 437, 2157, 357, 6, 33

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5654 [00:00<?, ? examples/s]

Map:   0%|          | 0/5353 [00:00<?, ? examples/s]

Map:   0%|          | 0/5353 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model with or without hyperparameter search

In [ ]:
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, recall_score, precision_score, f1_score, auc, precision_recall_curve, classification_report
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import mean_squared_error





# def compute_metrics(eval_pred):
#     from evaluate import load
#     predictions, labels = eval_pred
#     metric = load('mse')
#     return metric.compute(predictions=predictions, references=labels, squared=False)


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     if task != "stsb":
#         predictions = np.argmax(predictions, axis=1)
#     else:
#         predictions = predictions[:, 0]
#     return metric.compute(predictions=predictions, references=labels)

def metrics_report_cm(y_true, y_pred, output_dir, model_name, ts, save=True):

    cm = confusion_matrix(y_true, y_pred,normalize=None)
    cm_df = pd.DataFrame(cm, index=['SITB-', 'SITB+'], columns=['SITB-', 'SITB+'])
    cm_df_meaning = pd.DataFrame([['TN', 'FP'],['FN','TP']], index=['SITB-', 'SITB+'], columns=['SITB-', 'SITB+'])

    cm_norm = confusion_matrix(y_true, y_pred,normalize='all')
    cm_norm = (cm_norm*100).round(2)
    cm_df_norm = pd.DataFrame(cm_norm, index=['SITB-', 'SITB+'], columns=['SITB-', 'SITB+'])


    plt.rcParams['figure.figsize'] = [4,4]
    cm_display = ConfusionMatrixDisplay(cm_norm,display_labels=['SITB-', 'SITB+']).plot()
    # todo save

    if save:
        cm_df_meaning.to_csv(output_dir+f'cm_meaning_{model_name}_{ts}.csv')
        cm_df.to_csv(output_dir+f'cm_{model_name}_{ts}.csv')
        cm_df_norm.to_csv(output_dir+f'cm_norm_{model_name}_{ts}.csv')



    return cm_df_meaning, cm_df, cm_df_norm






def metrics_report_classification_report(y_true, y_pred,y_pred_proba_1, output_dir, model_name, ts):

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    np.set_printoptions(suppress=True)
    roc_auc = roc_auc_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)

    # calculate precision and recall for each threshold
    lr_precision, lr_recall, thresholds = precision_recall_curve(y_true, y_pred_proba_1)

    # TODO: add best threshold
    fscore = (2 * lr_precision * lr_recall) / (lr_precision + lr_recall)
    fscore[np.isnan(fscore)] = 0
    ix = np.argmax(fscore)
    best_threshold = thresholds[ix].item()


    pr_auc = auc(lr_recall, lr_precision)
    # AU P-R curve is also approximated by avg. precision
    # avg_pr = metrics.average_precision_score(y_true,y_pred_proba_1)

    sensitivity = recall_score(y_true,y_pred)
    specificity = tn / (tn+fp) # OR: recall_score(y_true,y_pred, pos_label=0)
    precision = precision_score(y_true,y_pred)

    results = pd.DataFrame([sensitivity, specificity,precision,f1, roc_auc,pr_auc, best_threshold],
                        index = ['Sensitivity', 'Specificity', 'Precision', 'F1', 'ROC AUC','PR AUC', 'Best th PR AUC']).T.round(2)

    results.to_csv(output_dir+f'results_{model_name}_{ts}.csv')
    return results



from matplotlib import pyplot as plt
from sklearn import metrics
from scipy.stats import pearsonr, spearmanr




from sklearn import metrics
def regression_report(y_test,y_pred,y_train=None,gridsearch=None, best_params=None,feature_vector=None,model_name=None,metrics_to_report = 'all', plot = True, save_fig_path = None, n = 'all', round_to = 2, figsize=(4,8), ordinal_ticks = True):
	'''
	metrics = {'all', ['MAE','RMSE','rho', 'Best parameters']
	}
	'''

	# Metrics
	# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
	rmse = metrics.mean_squared_error(y_test, y_pred, squared=False )
	mae = metrics.mean_absolute_error(y_test, y_pred)
	r2 = metrics.r2_score(y_test, y_pred)
	r, p = pearsonr(y_test, y_pred)
	rho, p = spearmanr(y_test, y_pred)

	results_dict = {
		'Features':feature_vector,
		'Estimator':model_name,
		'n':n,
		'y_train_min': np.min(y_train),
		'y_train_max': np.max(y_train),
		'RMSE':np.round(rmse,round_to ),
		'MAE':np.round(mae,round_to ),
		'R^2':np.round(r2,round_to ),
		'r':np.round(r,round_to ),
		'rho':np.round(rho,round_to ),
		'gridsearch':gridsearch,
		'Best parameters': str(best_params),
		}
	results = pd.DataFrame(results_dict, index=[model_name]).round(3)
	# results_all.append(results)

	if metrics_to_report == 'all' or ('RMSE per value' in metrics_to_report and 'MAE per value' in metrics_to_report):
		y_pred_test = {}
		y_pred_test['RMSE per value'] = []
		y_pred_test['MAE per value'] = []
		for value in np.unique(y_test):
			y_pred_test_i = [[pred,test] for pred,test in zip(y_pred,y_test) if test == value]
			y_pred_test[value] = np.array(y_pred_test_i)
			y_pred_i = [n[0] for n in y_pred_test_i]
			y_test_i = [n[1] for n in y_pred_test_i]
			rmse_i = metrics.mean_squared_error(y_test_i, y_pred_i, squared=False )
			mae_i = metrics.mean_absolute_error(y_test_i, y_pred_i)
			y_pred_test['RMSE per value'].append(np.round(rmse_i,round_to ))
			y_pred_test['MAE per value'].append(np.round(mae_i,round_to ))
		# print(y_pred_test['RMSE per value'])
		results_dict.update({
		'RMSE per value':f"{y_pred_test['RMSE per value']}",
		'MAE per value':f"{y_pred_test['MAE per value']}"
		})
		macro_avg_rmse = np.round(np.mean(y_pred_test['RMSE per value']), round_to)
		macro_avg_mae = np.round(np.mean(y_pred_test['MAE per value']), round_to)

		results_dict.update({
		'Macro avg. RMSE':f"{macro_avg_rmse}",
		'Macro avg. MAE':f"{macro_avg_mae}",
		})

		# metrics_to_report_2 = metrics_to_report.copy()
		# metrics_to_report_2.remove('RMSE') #redudant
		# metrics_to_report_2.remove('MAE') #redudant
		results = pd.DataFrame(results_dict, index=[model_name]) # replace with updated metrics
		# results = results[metrics_to_report_2]


	# Plot result for a regression task: true value vs predicted values
	# ============================================================
	plt.clf()
	plt.figure(figsize=figsize)  # Width=10 inches, Height=6 inches

	plt.style.use('default')  # Example of applying the 'ggplot' style
	plt.scatter(y_test, y_pred, alpha = 0.05)
	# plt.title(f"{feature_vector.capitalize().replace('_',' ')}")
	plt.xlabel('True values')
	plt.ylabel('Predicted values')


	ticks = list(np.unique(y_test))
	if ordinal_ticks and len(ticks)<12:
		plt.xticks(ticks=ticks,labels = [str(int(n)) for n in ticks])

	plt.tight_layout()
	if save_fig_path:
		plt.savefig(save_fig_path+'.png', dpi=300)
	# plt.show()
	return results



Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

### Custom Trainer to add class weights if you have class imbalance

In [ ]:

from torch import nn
from transformers import Trainer

from sklearn.utils.class_weight import compute_class_weight

class_weights =  compute_class_weight(class_weight = "balanced", classes= np.unique(y_train), y= y_train)
print(class_weights)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, device=model.device).float())
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

[0.99876347 1.00035386 1.00088511]


### define or download model and define training arguments

In [ ]:
%%time

# Finetuning no hyperparameter search:
# distilroberta 18m (4 epochs)
# roberta-base 35m (3 epochs)


def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    if regression:
      rmse = mean_squared_error(labels, predictions, squared=False)
      return {"rmse": rmse}
    else:
      if task != "stb":
          predictions = np.argmax(predictions, axis=1)
      else:
          predictions = predictions[:, 0]
      return metric.compute(predictions=predictions, references=labels)

model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,

  push_to_hub=False,

)


# specify model and model_init

# Regression also works with AutoModelForSequenceClassification but with num_labels =1 . Then the compute_metrics will be different
if do_hyperparameter_search:
    model = None
    def create_model_init():
        return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

    model_init = create_model_init


else:
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    model_init=None


def define_trainer(encoded_train_set, encoded_val_set, model=None, model_init=None, compute_metrics = None, imbalanced = imbalanced):
    if not imbalanced:
        trainer = Trainer(
            model = model,
            model_init = model_init,
            args = args,
            train_dataset=encoded_train_set,
            eval_dataset=encoded_val_set,
            tokenizer=tokenizer,
          compute_metrics=compute_metrics,
        )

    else:
        # here we use a CustomTrainer for class weights
        trainer = CustomTrainer(
            model = model,
            model_init = model_init,
            args = args,
            train_dataset=encoded_train_set,
            eval_dataset=encoded_val_set,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )
    return trainer



CPU times: user 2.68 ms, sys: 0 ns, total: 2.68 ms
Wall time: 5.63 ms


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-mrpc"` or `"huggingface/bert-finetuned-mrpc"`).

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

# Train and save model

In [ ]:
%%time

encoded_train_set = encoded_dataset["train"]
encoded_val_set = encoded_dataset[validation_key]

if do_hyperparameter_search:
    if hyperparameter_shards:
        encoded_train_dataset_shards = encoded_train_set.shard(index=1, num_shards=hyperparameter_shards)
        trainer = define_trainer(encoded_train_dataset_shards, encoded_val_set, model=model, model_init=model_init , compute_metrics = compute_metrics,imbalanced=imbalanced)
    else:
        trainer = define_trainer(encoded_train_set, encoded_val_set, model=model, model_init=model_init , compute_metrics = compute_metrics,imbalanced=imbalanced)
    best_run = trainer.hyperparameter_search(n_trials=hyperparameter_search_n_trials,
                                             direction=hyperparameter_search_direction,
                                             backend='optuna',
                                             hp_space = optuna_hp_space, # or will use default hp_space for whatever the backend is
                                             )
    print('==== best run:', best_run)
    print()

    # re define the trainer with the entire dataset
    trainer = define_trainer(encoded_train_set, encoded_val_set, model=model, model_init=model_init , compute_metrics = compute_metrics, imbalanced=imbalanced)
    for n, v in best_run.hyperparameters.items():
        setattr(trainer.args, n, v)

# train on entire training set for the last time
trainer.train()

ts_i = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')

if toy:
  output_dir_i = output_dir + f'results_{ts_i}_toy/'
else:
  output_dir_i = output_dir + f'results_{ts_i}/'
os.makedirs(output_dir_i, exist_ok=True)

trainer.save_model(output_dir_i)
# trainer.push_to_hub() # need to have added token above in TrainingArguments()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should pr

Epoch,Training Loss,Validation Loss,Rmse
1,No log,0.276627,0.525954
2,0.694300,0.235030,0.484799
3,0.226000,0.235892,0.485688


[I 2024-02-16 20:38:41,250] Trial 0 finished with value: 0.48568764328956604 and parameters: {'learning_rate': 2.086910709881257e-06, 'num_train_epochs': 3, 'weight_decay': 0.0001519847591139456}. Best is trial 0 with value: 0.48568764328956604.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of R

Epoch,Training Loss,Validation Loss,Rmse
1,No log,0.354791,0.595643
2,0.961100,0.329278,0.573827


[I 2024-02-16 21:01:53,662] Trial 1 finished with value: 0.5738271474838257 and parameters: {'learning_rate': 1.2673878926147143e-06, 'num_train_epochs': 2, 'weight_decay': 0.0006259136825629437}. Best is trial 0 with value: 0.48568764328956604.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of R

Epoch,Training Loss,Validation Loss,Rmse
1,No log,0.245697,0.495678
2,0.412300,0.238336,0.488197


[I 2024-02-16 21:25:08,278] Trial 2 finished with value: 0.4881969392299652 and parameters: {'learning_rate': 6.198716023432363e-06, 'num_train_epochs': 2, 'weight_decay': 2.0624087949578145e-07}. Best is trial 0 with value: 0.48568764328956604.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of R

Epoch,Training Loss,Validation Loss,Rmse
1,No log,0.393743,0.627489
2,0.304300,0.250181,0.500181


In [ ]:
# # Load
# from transformers import AutoModel, AutoConfig
# config = AutoModelForSequenceClassification.from_pretrained(output_dir_i)
# model = AutoModelForSequenceClassification.from_pretrained(output_dir_i, config=config)

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
# model_init=None


trainer = define_trainer(encoded_train_set, encoded_val_set, model=model, model_init=model_init, compute_metrics = compute_metrics, imbalanced = imbalanced)

In [ ]:
# best_params = f"weight_decay: {trainer.args.weight_decay}; num_train_epochs: {trainer.args.num_train_epochs}; learning_rate: {trainer.args.learning_rate}"
# best_params = "weight_decay: 6.8120818721882775e-06; num_train_epochs: 4; learning_rate: 1.094605203258915e-06"
# best_params


In [ ]:

best_params = str(list(best_run.hyperparameters.items()))
best_params

The best run was:



# Evaluate on test set

In [ ]:
import datetime


results = []




# Regression
if regression:
  feature_vector = None
  gridsearch = True

  y_pred = trainer.predict(test_dataset=encoded_dataset["test"]).predictions
  y_pred = y_pred.reshape(1,-1)[0]


  y_pred_df = pd.DataFrame(y_pred)
  y_pred_df.to_csv(output_dir_i+f'y_pred_{model_name}_gridsearch-{gridsearch}_{ts_i}.csv', index=False)
  path = output_dir_i + f'scatter_{model_name}_gridsearch-{gridsearch}_{ts_i}'

  # Performance
  results_i =regression_report(y_test,y_pred,y_train=y_train,
                              metrics_to_report = metrics_to_report,
                                gridsearch=gridsearch,
                              best_params=best_params,feature_vector=feature_vector,model_name=model_name, n=n, plot = True, save_fig_path = path, round_to = 2)




  results_i.to_csv(output_dir_i + f'results_{model_name}_gridsearch-{gridsearch}_{ts_i}.csv')
  display(results_i)
  results.append(results_i)
  results_df = pd.concat(results)
  results_df = results_df.reset_index(drop=True)
  results_df.to_csv(output_dir_i + f'results_{ts_i}.csv', index=False)

else:
  # Classification
  y_pred_proba = trainer.predict(test_dataset=encoded_dataset["test"]).predictions

  y_pred_proba = np.array(y_pred_proba)
  y_pred_proba_1 = y_pred_proba[:,1]
  y_pred = [np.argmax(n) for n in y_pred_proba

  test['y_pred_proba_0'] = y_pred_proba[:,0]
  test['y_pred_proba_1'] = y_pred_proba[:,1]
  test['y_pred'] = y_pred


  test.to_csv(output_dir_i+'test_predictions.csv', index=False)

  # y_pred = [int(np.argmax(n)) for n in y_pred_proba]
  print(metrics.classification_report(y_pred=y_pred, y_true=y_test))
  results = {}

  clf_report_sklearn = metrics.classification_report(y_test,y_pred, output_dict=False) #evaluate #different than the output of cross_validate() above.
  cm_df_meaning, cm_df, cm_df_norm = metrics_report_cm(y_test, y_pred, output_dir_results, model_name, ts, save=True)
  clf_report = metrics_report_classification_report(y_test,y_pred,y_pred_proba_1, output_dir_results, model_name, ts)
  # scores = cross_validate(pipe, X, y, scoring=['f1','precision', 'recall'], cv=cv, return_train_score=False) #train and evaluate
  results[model_name] = {
      'clf_report': clf_report,
      'cm_df_meaning': cm_df_meaning,
      'cm_df': cm_df,
      'cm_df_norm': cm_df_norm,
      'clf_report_sklearn': clf_report_sklearn

  }
  for k, v in results.get(model_name).items():
    print(k)
    if k != 'clf_report_sklearn':
      display(v)
    else:
      print(v)
    print()


In [ ]:
# trainer.evaluate(eval_dataset=encoded_dataset["test"])
# trainer.evaluate() # this is done on the validation set specificed in the Trainer()

In [ ]:
# load model:
#  - https://discuss.huggingface.co/t/how-to-save-my-model-to-use-it-later/20568/6
# - model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
# - then just load the trainer, but instead of training you just predict (here I'm training to fine tune, but if we load the fine tuned model , there's no need)


